In [1]:
import numpy as np
from scikits.odes.ode import ode

In [2]:
#Import local functions
sys.path.append('../functions')
from functions import nonlinearity, repression, activation

In [15]:
#Default information - user can change but mostly won't
#Constants (set to default)
kc=12.; km=10.; lam=1.93E-4; Vin=1.; e0=0.0467

#Integration conditions
y = np.array([2290.,0.,0.,0., 0., 0.])
t = np.linspace(0, 5e4, 100) 

In [4]:
#User specified information
X = 2 #Number of chemical species in pathway
E = 2 #Number of enzymes in pathway

#S = np.zeros([X, E])
S = np.array([[ -1, 0], [1, -1]]) #Stoichiometric matrix defining chemical reactions

I = np.zeros(X) #Input matrix defining inputs to pathway
I[0] = Vin #Inputs to pathway

T = [1] #Xs that can act as TFs

In [5]:
#Genetic parameter values for circuit
W = np.zeros([len(T), E, 3]) #K, theta, n are 3 dimensions

#Manually set initial conditions
for t in range(len(T)):
    for e in range(E):
        W[t][e] = [2., 0.1,1E-4]

In [6]:
#Architecture matrix
A = np.zeros([len(T), E, 3]) #3 methods of action at each locus for each TF

In [7]:
#Old code to generate kinetic equations
# for i in range(X): #Kinetic equations
#     ydot.append(I[i] - lam*y[i] + np.sum(S[i]*y[X:]*nonlinearity(y[i], kc, km)))
# ydot[0] = ydot[0] - e0*nonlinearity(y[0], kc, km) #Manually add constitutive metabolism

In [16]:
def generated_equation(t, y, ydot):
    '''Generate genetic equations from architecture matrix'''
    ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km) - y[2]*nonlinearity(y[0], kc, km)
    ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
    for e in range(E):
        ydot[e+X] = -lam*y[e+X] + np.sum([np.sum(A[t][e]*[activation(y[T[t]], W[t][e][2], W[t][e][1], W[t][e][0]), repression(y[T[t]], W[t][e][2], W[t][e][1], W[t][e][0]), W[t][e][2]]) for t in range(len(T))])
    ydot[E+X] = (I[0] - y[X]*nonlinearity(y[X-1], kc, km))**2 #J1
    ydot[E+X+1] = np.sum([np.sum([np.sum(A[t][e]*[activation(y[T[t]], W[t][e][2], W[t][e][1], W[t][e][0]), repression(y[T[t]], W[t][e][2], W[t][e][1], W[t][e][0]), 0]) for t in range(len(T))]) for e in range(E)]) #J2

In [17]:
#Verify results with upstream repression
def upstream_repression_sundials(t, y, ydot):
        ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km) - y[2]*nonlinearity(y[0], kc, km)
        ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
        ydot[2] = repression(y[1], k1, theta1, n1) - lam*y[2]
        ydot[3] = k2 - lam*y[3]
        ydot[4] = (Vin -  y[3]*nonlinearity(y[1], kc, km))**2
        ydot[5] = repression(y[1], k1, theta1, n1) 

n1, n2, theta1, theta2, k1, k2 = [2.,2., 0.1, 0.1, 1E-4, 1E-4]
ydot_dc = np.zeros(6)
upstream_repression_sundials(t, y, ydot_dc)

A = np.zeros([len(T), E, 3]) #3 methods of action at each locus for each TF
A[0][0] = [0, 1, 0]
A[0][1] = [0, 0, 1]

ydot = np.zeros(6)
generated_equation(t, y, ydot)

ydot-ydot_dc

array([0., 0., 0., 0., 0., 0.])

In [18]:
#Verify results with downstream activation
def downstream_activation_sundials(t, y, ydot):
        ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km) - y[2]*nonlinearity(y[0], kc, km)
        ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
        ydot[2] = k1 - lam*y[2]
        ydot[3] = activation(y[1], k2, theta2, n2) - lam*y[3]
        ydot[4] = (Vin -  y[3]*nonlinearity(y[1], kc, km))**2
        ydot[5] = activation(y[1], k2, theta2, n2)

n1, n2, theta1, theta2, k1, k2 = [2.,2., 0.1, 0.1, 1E-4, 1E-4]
ydot_dc = np.zeros(6)
upstream_repression_sundials(t, y, ydot_dc)



ydot = np.zeros(6)
generated_equation(t, y, ydot)

ydot-ydot_dc

array([0., 0., 0., 0., 0., 0.])

In [19]:
#Verify results with dual control
def dual_control_sundials(t, y, ydot):
        ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km) - y[2]*nonlinearity(y[0], kc, km)
        ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
        ydot[2] = repression(y[1], k1, theta1, n1) - lam*y[2]
        ydot[3] = activation(y[1], k2, theta2, n2) - lam*y[3]
        ydot[4] = (Vin -  y[3]*nonlinearity(y[1], kc, km))**2
        ydot[5] = repression(y[1], k1, theta1, n1) + activation(y[1], k2, theta2, n2)

n1, n2, theta1, theta2, k1, k2 = [2.,2., 0.1, 0.1, 1E-4, 1E-4]
ydot_dc = np.zeros(6)
dual_control_sundials(t, y, ydot_dc)


A = np.zeros([len(T), E, 3]) #3 methods of action at each locus for each TF
A[0][0] = [0, 1, 0]
A[0][1] = [1, 0, 0]

ydot = np.zeros(6)
generated_equation(t, y, ydot)

ydot-ydot_dc

array([0., 0., 0., 0., 0., 0.])

In [12]:
#Default information - user can change but mostly won't
#Constants (set to default)
kc=12.; km=10.; lam=1.93E-4; Vin=1.; e0=0.0467

#Integration conditions
y = np.array([2290.,0.,1.,0.])
t = np.linspace(0, 5e4, 100) 